In [1]:
from datasets import load_dataset

#dataset_name = "timdettmers/openassistant-guanaco" ###Human ,.,,,,,, ###Assistant

dataset = load_dataset("tatsu-lab/alpaca")
dataset = dataset['train']
dataset = dataset.select(range(100))

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /home/arjun/NewPytorchEnv/bin/python3.10: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizers and quantization routines!!
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 00


/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('0'), PosixPath('1')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('@/tmp/.ICE-unix/1923,unix/PC'), PosixPath('local/PC')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('vs/workbench/api/node/extensionHostProcess')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/etc/xdg/xdg-ubuntu')}
  warn(

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [4]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

In [5]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 4
gradient_accumulation_steps = 24
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 3
learning_rate = 1e-4
max_grad_norm = 0.3
max_steps = 100
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [6]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

[codecarbon INFO @ 20:32:01] [setup] RAM Tracking...
[codecarbon INFO @ 20:32:01] [setup] GPU Tracking...
[codecarbon INFO @ 20:32:01] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:32:01] [setup] CPU Tracking...
[codecarbon WARNING @ 20:32:01] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 20:32:03] We saw that you have a AMD Ryzen 7 5700G with Radeon Graphics but we don't know it. Please contact us.
[codecarbon INFO @ 20:32:03] CPU Model on constant consumption mode: AMD Ryzen 7 5700G with Radeon Graphics
[codecarbon INFO @ 20:32:03] >>> Tracker's metadata:
[codecarbon INFO @ 20:32:03]   Platform system: Linux-6.2.0-31-generic-x86_64-with-glibc2.35
[codecarbon INFO @ 20:32:03]   Python version: 3.10.12
[codecarbon INFO @ 20:32:03]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 20:32:03]   Available RAM : 15.400 GB
[codecarbon INFO @ 20:32:03]   CPU count: 16
[codecarbon INFO @ 20:32:03]   CPU model: AMD Ryzen 7 5700G with Radeon Graphics
[c

In [7]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [ ]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: arjun_g_ravi. Use `wandb login --relogin` to force relogin


  0%|          | 0/100 [00:00<?, ?it/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
[codecarbon INFO @ 20:32:27] Energy consumed for RAM : 0.000024 kWh. RAM Power : 5.774871826171875 W
[codecarbon INFO @ 20:32:27] Energy consumed for all GPUs : 0.001596 kWh. Total GPU Power : 382.971 W
[codecarbon INFO @ 20:32:27] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:32:27] 0.001797 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:32:42] Energy consumed for RAM : 0.000048 kWh. RAM Power : 5.774871826171875 W
[codecarbon INFO @ 20:32:42] Energy consumed for all GPUs : 0.003211 kWh. Total GPU Power : 387.702 W
[codecarbon INFO @ 20:32:42] Energy consumed for all CPUs : 0.000354 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:32:42] 0.003613 kWh of electricity used since the beginning.
[co

{'loss': 90467328.0, 'learning_rate': 0.0001, 'epoch': 2.88}


[codecarbon INFO @ 20:33:12] Energy consumed for RAM : 0.000096 kWh. RAM Power : 5.774871826171875 W
[codecarbon INFO @ 20:33:12] Energy consumed for all GPUs : 0.006441 kWh. Total GPU Power : 388.663 W
[codecarbon INFO @ 20:33:12] Energy consumed for all CPUs : 0.000708 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:33:12] 0.007245 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:33:27] Energy consumed for RAM : 0.000120 kWh. RAM Power : 5.774871826171875 W
[codecarbon INFO @ 20:33:27] Energy consumed for all GPUs : 0.008045 kWh. Total GPU Power : 385.05 W
[codecarbon INFO @ 20:33:27] Energy consumed for all CPUs : 0.000885 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:33:27] 0.009050 kWh of electricity used since the beginning.


KeyboardInterrupt: 

[codecarbon INFO @ 20:33:42] Energy consumed for RAM : 0.000144 kWh. RAM Power : 5.774871826171875 W
[codecarbon INFO @ 20:33:42] Energy consumed for all GPUs : 0.008654 kWh. Total GPU Power : 146.26500000000001 W
[codecarbon INFO @ 20:33:42] Energy consumed for all CPUs : 0.001063 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:33:42] 0.009861 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:33:57] Energy consumed for RAM : 0.000168 kWh. RAM Power : 5.774871826171875 W
[codecarbon INFO @ 20:33:57] Energy consumed for all GPUs : 0.008825 kWh. Total GPU Power : 41.112 W
[codecarbon INFO @ 20:33:57] Energy consumed for all CPUs : 0.001240 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 20:33:57] 0.010233 kWh of electricity used since the beginning.
[codecarbon INFO @ 20:34:12] Energy consumed for RAM : 0.000192 kWh. RAM Power : 5.774871826171875 W
[codecarbon INFO @ 20:34:12] Energy consumed for all GPUs : 0.008982 kWh. Total GPU Power : 37.787 W
[codecarbon INFO @

In [ ]:
save_loc1 = '/home/arjun/Documents/ModelSaves/llama2-test'

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(save_loc1)
trainer.save_model(save_loc1)

In [ ]:
bug

Inference

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig.from_pretrained(save_loc1)
model = get_peft_model(model, lora_config)

In [ ]:
text = "###Human: Give me tips to be healthy### Assistant:"
device = "cuda:0"

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=200)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))